# spotahome main table

In [ ]:
# import all libraries that are needed

import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing

import pandas as pd             # pandas for data frame

import math                     # for math methods
import time                     # for sleep timer

from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine

import datetime as dt           # for the csv file with the current date and time

# libraries that are not needed right now
import re
#import json
#import numpy as np
#from zipfile import *


# declaration of functions to get specific information from the website

# Creating a function to get all the descriptions
def get_description(bs):
    # find all the descriptions and save them to an empty list
    lst_name = []
    descriptions = bs.find_all(
        class_='homecard-content__title__HomecardContent___OmV4c homecard-content__title--rebranding-style__HomecardContent___OmV4c')
    # iterate over the descriptions to get the text and strip the strings and save them in a list
    for description in descriptions:
        lst_name.append(
            description.get_text()
                .strip()
        )
    return lst_name


# Creating a function to get all the housing types
def get_housing(bs):
    # find all the housing types and save them to an empty list
    lst_name = []
    housings = bs.find_all(
        class_='homecard-content__type__HomecardContent___OmV4c homecard-content__type--rebranding-style__HomecardContent___OmV4c')
    # iterate over the housing types to get the text and strip the strings and save them in a list
    for housing in housings:
        lst_name.append(
            housing.get_text()
                .strip()
        )
    return lst_name


# Creating a function to get all the available dates
def get_available(bs):
    # find all the available dates and save them to an empty list
    lst_name = []
    availables = bs.find_all(
        class_='homecard-content__available-from__HomecardContent___OmV4c homecard-content__available-from--rebranding-style__HomecardContent___OmV4c')
    # iterate over the available appartements to get the text and strip the string and save them in a list
    for available in availables:
        lst_name.append(
            available.get_text()
                .strip()
                .replace('From ', '')
        )
    return lst_name


# Creating a function to get all the prices
def get_price(bs):
    # find all the prices and save them to an empty list
    lst_name = []
    prices = bs.find_all(class_='price__Price___OmV4c')
    # iterate over the prices to get the text and strip the strings and save them in a list
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
                .replace('£', '')
                .split('-')[0]
        )
    return lst_name


# Creating a function to get all the prices per period
def get_prices_period(bs):
    # find all the aprices per period and save them to an empty list
    lst_name = []
    prices_period = bs.find_all(
        class_='price-monthly__Price___OmV4c price-monthly--rebranding-style__Price___OmV4c')
    # iterate over the prices per period to get the text and strip the string and save them in a list
    for price_period in prices_period:
        lst_name.append(
            price_period.get_text()
                .strip()
                .replace('/', '')
        )
    return lst_name


# Creating a function to get all the ID's
def get_ids(bs):
    # find all the prices and save them to an empty list
    lst_name = []
    ids = bs.find_all(class_='l-list__item')
    # iterate over the prices to get the text and strip the strings and save them in a list
    for id in ids:
        lst_name.append(
            id.get('data-homecard-scroll')
                .strip()
        )
    return lst_name


# Create dictionary in which every location ID gets assigned a location name
location_dict = {219: 'Lambeth',
                 231: 'Hammersmith and Fulham',
                 232: 'Kensington and Chelsea',
                 233: 'City of Westminster',
                 234: 'Camden',
                 235: 'Tower Hamlets',
                 236: 'Islington',
                 237: 'Hackney',
                 241: 'City of London'
                 }


# Creating a function to get the search result from all pages
# the website spotahome shows 60 search results per page. To iterate trough all the pages, we get the information how many search results are there, then divide it by 60 and round it up to get the number of pages.
def page_results(property_type, location):
    # get the url from the website with the property type and the location as a variable to iterate trough it
    page = requests.get(
        f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}?areaId[]={location}')
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # Extracting the total number of search results
    results = bs.find_all('h1', {'class': 'search-title__title'})

    # define the variable result_text in case the first search gives us no results
    result_text = 0

    # iterate over the results to get the text and strip the string
    for result in results:
        result_text = result.find("strong").get_text().strip()

    # convert the extracted string to an integer to perform mathematical operations
    result_converted = int(result_text)

    # divide the converted result by 60 since one pages shows 60 results and round it up to get the number of pages
    page_site = result_converted / 60
    page_site = math.ceil(page_site)

    # convert the number of pages from a float to an integer to iterate trough the pages
    page_converted = int(page_site)

    # create an empty data frame to store the results from every loop cycle
    df_search = pd.DataFrame()

    # split the url to get access to the part where the page is definde
    begin = f'https://www.spotahome.com/s/london--uk/for-rent:{property_type}'
    end = f'?areaId[]={location}'

    # range is including in the beginning and excluding in the end so we add plus 1 to iterate through all calculated pages
    page_converted = page_converted + 1

    # for loop to get the page numbers
    for page_number in range(page_converted):
        # sleep timer to reduce the traffic for the server
        time.sleep(0.5)

        # get the url from the website with the property type, the location and the page number as a variable to iterate trough it
        # middle =
        page = requests.get(begin+f'/page:{page_number}'+end)
        html = page.content
        bs = BeautifulSoup(html, 'html.parser')

        # Create a dictionary to store the results from every loop cycle.
        # The keys are the column names and the values are the functions we created before.
        # The functions are called with the beautiful soup object as a parameter.
        spotahome_dict = {
            'platform_id': get_ids(bs),
            'platform': 'spotahome',
            'neighborhood': location_dict[location],
            'property_type': property_type,
            'housing_type': get_housing(bs),
            'price_pcm': get_price(bs),
            'title': get_description(bs),
            'furnished': 'furnished',
            'available_from': get_available(bs),
        }
        # the ditionary is stored in a dataframe
        df_page = pd.DataFrame(data=spotahome_dict)

        # the temporary data frame gets appended to the data frame we created earlier outside the for loop
        # for every iteration, the data frame page stores the results in the data frame search
        #df_search = df_search.append(df_page)
        df_search = pd.concat([df_search, df_page], axis=0, ignore_index=True)
    # the data frame search gets returned to the for loop to access it outside the function
    return (df_search)


# creating a list with different property types given from the website
property_types = ['studios', 'apartments/bedrooms:1',
                  'apartments/bedrooms:2', 'apartments/bedrooms:3', 'apartments/bedrooms:3more']
# creating a list with the different location IDs given from the website
locations = [219, 231, 232, 233, 234, 235, 236, 237, 241]

# creating an empty data frame
df_complete = pd.DataFrame()
# for loop to get the different property types
for property_type in property_types:
    # for loop to get the different locations
    for location in locations:
        # append the result from data frame search by calling the function page_results with the property type and the location as a parameter to data frame complete
        df_complete = pd.concat([df_complete, page_results(
            property_type, location)], axis=0, ignore_index=True)


In [ ]:
# display the data frame
display(df_complete)

In [ ]:
# get information about the data frame
# results for 5 different property types and 9 different locations with 60 search results per page is 844 on 22.09.2022 10:00 h
df_complete.info()

## details splitted in separate columns

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


df_details_complete = pd.DataFrame()

for idx, row in df_complete.iterrows():
    time.sleep(0.5)
    page = f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}"
    website = requests.get(page)
    results = BeautifulSoup(website.content, 'html.parser')

    details = results.find(
        'div', class_='property-title__details').find_all('span')

    details_lst = (detail.get_text() for detail in details)
    details_lst = [detail.strip() for detail in details_lst]

    details_lst.pop(0)

    details_lst = [i.split(' ', 1) for i in details_lst]

    row_dict = {}

    for value_key_tuple in details_lst:
        new_key_value = {'id': row['platform_id']}
        row_dict.update(new_key_value)
        key = value_key_tuple[1]
        value = value_key_tuple[0]
        row_dict[key] = value

    df_details = pd.DataFrame(data=row_dict, index=[0])
    df_details_complete = pd.concat([df_details_complete, df_details])


In [ ]:
df_details_complete

# export to DBeaver

In [ ]:
# import the data frame to DBeaver

# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'spotahome_df_complete'

# import the table to sql
if engine!=None:
    try:
        df_complete.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
# import the data frame to DBeaver

# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'spotahome_df_details_complete'

# import the table to sql
if engine!=None:
    try:
        df_details_complete.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

# spotahome table for every detail page

In [ ]:
# def get_details(bs):
#     details = bs.find_all(class_='property-title__details')
#     details_lst = (detail.get_text() for detail in details)
#     details_lst = [detail.strip() for detail in details_lst]
#     return details_lst


# df_details = pd.DataFrame()

# for idx, row in df_complete.iterrows():
#     page=requests.get(
#         f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}")
#     html=page.content
#     bs=BeautifulSoup(html, 'html.parser')

#     details_dict={
#         'id': row['platform_id'],
#         'housing': row['housing_type'].lower() + 's',
#         'details': get_details(bs),
#     }
#     df_url_housing_id=pd.DataFrame(data=details_dict)
#     df_details=pd.concat(
#         [df_details, df_url_housing_id], axis=0, ignore_index=True)


In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import re
# import pandas as pd


# def get_details(page):
#     website = requests.get(page)
#     results = BeautifulSoup(website.content, 'html.parser')

#     details = results.find(
#         'div', class_='property-title__details').find_all('span')

#     details_lst = (detail.get_text() for detail in details)
#     details_lst = [detail.strip() for detail in details_lst]

#     details_lst.pop(0)

#     details_lst = [i.split(' ', 1) for i in details_lst]

#     row_dict = {}

#     for value_key_tuple in details_lst:
#         new_key_value = {'id': row['platform_id']}
#         row_dict.update(new_key_value)
#         key = value_key_tuple[1]
#         value = value_key_tuple[0]
#         row_dict[key] = value

#     return row_dict


# df_details_complete = pd.DataFrame()

# for idx, row in df_complete.iterrows():
#     page = f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}"
#     website = requests.get(page)
#     results = BeautifulSoup(website.content, 'html.parser')

#     df_details = pd.DataFrame(get_details(page), index=[0])
#     df_details_complete = pd.concat([df_details_complete, df_details])


## test code for one property - NOT WORKING

In [ ]:
# def get_details(bs):
#     lst_name = []
#     details = bs.find_all(class_='property-title__details')
#     for detail in details:
#         lst_name.append(
#             details.get_text()
#                 .strip()
#         )
#     return lst_name


# def get_details_strong(bs):
#     lst_name = []
#     details_strong = bs.find_all(class_='property-title__details')
#     for detail_strong in details_strong:
#         lst_name.append(
#             details_strong.get('strong').strip()
#         )
#     return lst_name

# page = requests.get(
#     'https://www.spotahome.com/london/for-rent:apartments/604187')
# html = page.content
# bs = BeautifulSoup(html, 'html.parser')

# details_dict = {
#     'details': get_details(bs),
#     'details_only_int': get_details_strong(bs)
# }
# df_url_housing_id = pd.DataFrame(data=details_dict)
# df_details = pd.concat(
#     [df_details, df_url_housing_id], axis=0, ignore_index=True)

In [ ]:
# display(df_details)

## test code for all properties with separated information

In [ ]:
# def get_details(bs):
#     #details = bs.find_all(class_='property-title__details')
#     #details_lst = (detail.text.re.compile('m2') for detail in details)
#     details = bs.find_all('div', class_='property-title__details').find_all('span')
#     details_lst = (detail.get_text() for detail in details)
#     details_lst = [detail.strip() for detail in details_lst]
#     return details_lst


# df_details = pd.DataFrame()

# for idx, row in df_complete.iterrows():
#     page = requests.get(
#         f"https://www.spotahome.com/london/for-rent:{row['housing_type'].lower() + 's'}/{row['platform_id']}")
#     html = page.content
#     bs = BeautifulSoup(html, 'html.parser')

#     details_dict = {
#         #'id': row['platform_id'],
#         #'housing': row['housing_type'].lower() + 's',
#         'details': get_details(bs),
#     }
#     df_url_housing_id = pd.DataFrame(data=details_dict)
#     df_details = pd.concat(
#         [df_details, df_url_housing_id], axis=0, ignore_index=True)



In [ ]:
# display(df_details)

In [ ]:
# # create a .csv file with the current date and time
# today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
# df_complete.to_csv('spotahome_{}.csv'.format(today), sep='\t')